# Introduction
This module implements basic LSTM architecture by building up on the implementation of vanilla RNN


In [1]:
import numpy as np 
import matplotlib.pyplot as plt
%load_ext autoreload
%matplotlib inline

## Set up and Read Data

In [2]:
data = open('text.txt','r').read().split()
vocab = list(set(data))
vocab_size = len(vocab)
print("There are %d uniques vocabs."% len(vocab))
#Create dictionary of indices for vocabs
char_to_indx = {char.lower():indx for indx,char in enumerate(vocab)}
indx_to_char = {indx:char.lower() for indx,char in enumerate(vocab)}

There are 83 uniques vocabs.


## Set up Parameters

In [3]:
num_hidden = 200
seq_length = 20 #Equivalent to time step
learning_rate = 2e-1
#Remember that the input vector has the length of the sequence
#And in LSTM, the vectors are "stacked" together at each step
z_size     = num_hidden + vocab_size

## Define Placeholders


In [4]:
#Recall that in LSTM, the general equation for each cell/timestep is:
# (i f o g).T = (sigmoid =sigmoid sigmoid tanh).T*W_matrix*(ht-1 xt).T
# Backprop must go through one of each gate, so we need a weight matrix for each of them
Wi = np.random.randn(num_hidden, z_size)*0.1
bi = np.zeros((num_hidden, 1))

Wf = np.random.randn(num_hidden, z_size)*0.1
bf = np.zeros((num_hidden, 1))

Wo = np.random.randn(num_hidden, z_size)*0.1
bo = np.zeros((num_hidden, 1))

Wg = np.random.randn(num_hidden, z_size)*0.1
bg = np.zeros((num_hidden, 1))

#Observe the dimensions of the matrices follow intuition by matrix multiplication rule
Wy = np.random.randn(vocab_size,num_hidden)*0.1
by = np.zeros((vocab_size, 1))

In [5]:
#Define gradients 
dWi = np.zeros_like(Wi)
dbi = np.zeros_like(bi)

dWf = np.zeros_like(Wf)
dbf = np.zeros_like(bf)

dWo = np.zeros_like(Wo)
dbo = np.zeros_like(bo)

dWg = np.zeros_like(Wg)
dbg = np.zeros_like(bg)

dWy = np.zeros_like(Wy)
dby = np.zeros_like(by)

In [6]:
#Helpfer functions
def sigmoid(x): 
    '''Assume that x is a a vector of dimension (some_num,1)'''
    return 1/(1 + np.exp(-x))

def dsigmoid(dx):
    return dx*(1-dx)

def softmax(x):
    '''Assume that x is a a vector of dimension (some_num,1)'''
    return np.exp(x)/np.sum(np.exp(x))

def dtanh(dx):
    return 1-dx*dx

In [7]:
def forward(inputs, hprev, cprev):
    '''
        Inputs should be a vector/array of that correspond to the indices
        *Note: compare this with the parameters of VRNN to see the difference in architecture
    '''
    #Test dimensions before running 
    assert inputs.shape == (vocab_size,1)
    assert hprev.shape  == (num_hidden,1)
    assert cprev.shape  == (num_hidden,1)
    z = np.row_stack([hprev, inputs])
    f = sigmoid(np.dot(Wf,z) + bf)
    
    i = sigmoid(np.dot(Wi,z) + bi)
    
    g = np.tanh(np.dot(Wg,z) + bg) #Should be equivalent to C_bar in the source code, and also tanh
   
    c_t = (f*cprev) + (i*g)
    
    o = sigmoid(np.dot(Wo,z) + bo)
    h_t = o*np.tanh(c_t)
    
    #Output probability of this time cell
    y = np.dot(Wy,h_t) + by
    p = softmax(y)
    
    return z,i,f,o,g,c_t,h_t,y,p

In [8]:
def backward(targets, dhnext, dcnext, cprev, z, i, f, o, g, c_t, h_t, y, p):
    '''Targets should be a vector of intergers
    Note that since the forward pass invoves h and c, the parameters here
    involve dhnext and dcnext
    '''
    global dWi, dWf, dWo, dWg, dWy
    global dbi, dbf, dbo, dbg, dby
    
    #Verify dimension compatibility 
    assert z.shape == (z_size, 1)
    assert y.shape == (vocab_size,1)
    assert p.shape == (vocab_size,1)
    
    for item in [i,f,o,g,c_t,h_t]: 
        assert item.shape == (num_hidden,1), "Shape is {} but not {}".format(item.shape, (num_hidden,1))
    
    dy = np.copy(p)
    dy[targets] -= 1
    dWy += np.dot(dy,h_t.T)
    dby += dy
    
    dh = np.dot(Wy.T, dy)
    dh += dhnext
    
    do = dh*np.tanh(c_t)
    do = dsigmoid(c_t) * do
    #Update dWo
    dWo += np.dot(do, z.T)
    dbo += do
    #Update dWg
    dc = np.copy(dcnext)
    dc += dh * o * dtanh(np.tanh(c_t))
    dg = dc * i
    dg = dg * dtanh(g)#Note: equivalent to dg
    dWg += np.dot(dg, z.T)
    dbg += dg
    #Update dWi
    di = dc * g 
    di = di * dsigmoid(i)
    dWi += np.dot(di, z.T)
    dbi += di
    #Update dWf
    df = dc * cprev
    df = df * dsigmoid(f)
    dWf += np.dot(df, z.T)
    dbf += df
    #update dz (remember that z is also one of the input)
    dz = np.dot(Wi.T,di) + np.dot(Wf.T,df) + np.dot(Wo.T,do) + np.dot(Wg.T,dg)
    dhprev = dz[:num_hidden,:] #Gradient for h should only have dimension of h
    dcprev  = dc * f
    return dhprev, dcprev

In [9]:
#TEST CODE
test_input = np.zeros((vocab_size,1))
test_input[np.random.randint(0,vocab_size-1,size=1)] = 1
test_hprev = np.random.randn(num_hidden,1)*1e-2
test_cprev = np.random.randn(num_hidden,1)*1e-2

In [10]:
#Test FORWARD
a,b,c,d,e,f,g,h,i =forward(test_input, test_hprev, test_cprev)
print("a:({}), b:({}), c:({}),d:({}),e:({});,f:({}),g:({}),h:({}),i:({}))".format(a.shape,b.shape,c.shape
                                                                   ,d.shape,e.shape,f.shape
                                                                  ,g.shape,h.shape, i.shape))

a:((283, 1)), b:((200, 1)), c:((200, 1)),d:((200, 1)),e:((200, 1));,f:((200, 1)),g:((200, 1)),h:((83, 1)),i:((83, 1)))


In [11]:
#Test BACKWARD
m, n = backward(5, test_hprev, test_cprev, test_cprev, a,b,c,d,e,f,g,h,i)
print("m shape: ({}), n shape: ({})".format(m.shape, n.shape))
del a,b,c,d,e,f,g,h,i,m,n

m shape: ((200, 1)), n shape: ((200, 1))


In [12]:
def one_pass(inputs, targets, hprev, cprev):
    '''
    Perform one forward step and one backward step through the sequence
    '''
    x_s, z_s, i_s, f_s, o_s, g_s, c_s, h_s, t_s, y_s, p_s =  {},{},{},{},{},{},{},{},{},{},{} 
    h_s[-1] = hprev
    c_s[-1] = cprev
    loss = 0
    #FORWARD pass
    for t in range(len(inputs)): 
        x_s[t] = np.zeros((vocab_size,1))
        x_s[t][inputs[t]] = 1
        z_s[t], i_s[t], f_s[t], o_s[t], g_s[t], c_s[t], h_s[t], y_s[t], p_s[t]  \
            = forward(x_s[t], h_s[t-1], c_s[t-1])
        loss += -np.log(p_s[t][targets[t],0]) 
    #BACKWARD pass
    #Reset all gradietn to 0 for the run
    for dparam in [dWi, dWf, dWo, dWg, dWy, dbi, dbf, dbo, dby, dbg]:
        dparam.fill(0)
        
    dhnext = np.zeros_like(h_s[0])
    dcnext = np.zeros_like(c_s[0])
    for t in reversed(range(len(inputs))):
        dhnext, dcnext = backward(targets=targets[t],dhnext=dhnext,dcnext=dcnext, cprev=c_s[t-1], \
                                 z=z_s[t], i=i_s[t],f=f_s[t],o=o_s[t],g=g_s[t],c_t=c_s[t],h_t=h_s[t],\
                                 y=y_s[t],p=p_s[t])
    
    #Perform gradient clipping
    for dparam in [dWi, dWf, dWo, dWg, dWy, dbi, dbf, dbo, dby, dbg]:
        np.clip(dparam,-1,1,out=dparam)

    return loss, h_s[len(inputs) - 1], c_s[len(inputs) - 1]

In [13]:
def sample(hprev, cprev, seed_ix, sentence_length):
    '''Randomly sample from the network sometimes. 
    '''
    x = np.zeros((vocab_size,1))
    x[seed_ix] = 1
    indices = []
    
    h = hprev
    c = cprev
    
    #Loop through n time steps
    for t in range(sentence_length):
        _, _, _, _, _, _, _, _, p = forward(inputs=x, hprev=h, cprev=c)
        idx = np.random.choice(range(vocab_size),p=p.ravel())#choose a random index in the output
        x = np.zeros((vocab_size,1))
        x[idx] = 1
        indices.append(idx)
    return indices

In [14]:
#Create memory for AdamGrad
mWi = np.zeros_like(Wi)
mWf = np.zeros_like(Wf)
mWo = np.zeros_like(Wo)
mWg = np.zeros_like(Wg)
mWy = np.zeros_like(Wy)

mbi = np.zeros_like(bi)
mbf = np.zeros_like(bf)
mbo = np.zeros_like(bo)
mbg = np.zeros_like(bg)
mby = np.zeros_like(by)

In [15]:
n, p = 0, 0
smooth_loss = -np.log(1.0/vocab_size)*seq_length
while n < 1e4: 
    try: 
        if p + seq_length + 1 >= len(data) or n==0:
            hprev = np.zeros((num_hidden,1))
            cprev = np.zeros((num_hidden,1))
            p = 0 
        
        inputs = [char_to_indx[ch.lower()] for ch in data[p:p+seq_length]]
        targets = [char_to_indx[ch.lower()] for ch in data[p+1:p+seq_length+1]]
        
        loss, hprev, cprev = one_pass(inputs, targets, hprev, cprev)
        smooth_loss = smooth_loss * 0.999 + loss * 0.001
        
        if n % 1e2 == 0:
            sample_ix = sample(hprev, cprev, inputs[0], 50)
            txt = ' '.join(indx_to_char[ix] for ix in sample_ix)#Add the next one
            print('----\n %s \n----'% (txt, ))

        
        if n% 1e2 == 0:
            print('iter %d, loss: %f'% (n, smooth_loss)) # print progress
            
        # perform parameter update with Adagrad
        for param, dparam, mem in zip([Wi, Wf, Wo, Wg, Wy, bi, bf, bo, bg, by], 
                            [dWi, dWf, dWo, dWg, dWy, dbi, dbf, dbo, dbg, dby], 
                            [mWi, mWf, mWo, mWg, mWy, mbi, mbf, mbo, mbg, mby]):
            mem += dparam * dparam
            param += -(learning_rate * dparam / np.sqrt(mem + 1e-8)) # adamgrad update
            
        p+=seq_length    
        n+=1
    except KeyboardInterrupt:
        print("User terminated process")
        break

----
 places i've (love, one slipped ever the uh and feel say swearing 'round just not anymore) picture picture knees last (love, wander we slipped one (love, insecure it phone it doesn't off uh do hear and would ever and on anymore, i'm hear had? anything said your i've makes doesn't 
----
iter 0, loss: 88.376784
----
 because because because i that i'm one still because just because because i i because just just just your i anything i yours" i just i i because that because because i run i your just feel ever just just best because when because i'm because 'round because the because 
----
iter 100, loss: 91.514281
----
 last anymore am so doesn't anymore, you so just best you just am never you doesn't my my know just ever just doesn't bad? never say just just the "i'm just knees just know because the that just go just just mind we am the i your say lying doesn't 
----
iter 200, loss: 91.654930
----
 when never well, anymore you myself because just because again run never because becau

----
 the ever bad? name again i'm you the just am because doesn't would you never missing well, doesn't never say ever on swearing i'll i'll because not change i'll lying say i say say on swearing you doesn't change because anything know and knees again when go say not you're 
----
iter 2900, loss: 68.670031
----
 because you never love the anything mind you're ever when ever the never myself lying to just because song, ever because myself again hear uh know doesn't anything you on you i ever when i'm on myself again change (love, again best more hoping more insecure am you you said 
----
iter 3000, loss: 68.310495
----
 because heard you you i am do change that say swearing that i'm you're the you accidentally i say you say bad? and i'd "i'm yours" yours" change to to when i i swearing "i'm bad? say to mean to i'll you had? yours" you doesn't to you so you 
----
iter 3100, loss: 67.691264
----
 my that because insecure because i'm bad? just because i to just because you're missing an

----
 uh uh not just the makes just because makes i and insecure said we uh uh uh knees insecure i uh uh uh uh uh me and into uh uh and said insecure insecure i've would uh uh uh knees just run into uh not your me insecure am i 
----
iter 5700, loss: 47.061834
----
 am i'm doesn't ever song, i'm am am just never ever i'll missing doesn't missing say i've i'm "i'm and never i'm lying my yours" i'm missing my had? still the i'm doesn't never the just because ever my hear just say lying i'm just i'm had? ever never say 
----
iter 5800, loss: 46.074324
----
 just because i would said the know i mean you so i to not i i i my i i know i'll love on i ever ever doesn't to i you so that would the i i 'round said the feel mean again i accidentally i i i know i 
----
iter 5900, loss: 44.969763
----
 know ever ever just because to when again say do when never doesn't i myself on to because say me you ever when i when doesn't mean swearing again because yours" ever ever my i'd when you do anything 

----
 you're my phone we anymore say to myself when you're my you're my phone am i'm on my you're my knees that i've i'm i lying go anymore, i never ever picture again hear uh love) we just because say again that hoping that i best i've to to again 
----
iter 8600, loss: 42.762825
----
 when because mean i just because anything i'm to i you say i lying mean just because ever to would when (love, when myself you i you say i lying because just because just because doesn't mean your more i and the i'd you one just because again when (love, 
----
iter 8700, loss: 42.482599
----
 because i yours" i'm best missing just because me took i i to when swearing you still slipped missing know am i (love, am i lying just because again when you i'm you say just because again when when phone love doesn't mean ever i say that i lying to 
----
iter 8800, loss: 42.123531
----
 doesn't the you so say you're am your song, just the so i doesn't doesn't the again it slipped still doesn't doesn't lying i'll w